In [1]:
import torch
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model
import evaluate
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, BitsAndBytesConfig, AutoModelForSeq2SeqLM

import matplotlib.pyplot as plt
from utils import prepare_prompt, print_trainable_parameters
import transformers
from tqdm import tqdm

import numpy as np
import json

In [2]:
dataset_raw = load_dataset('json', data_files='dataset_llm_generated.json')
dataset = dataset_raw.select_columns(["text", "summary"])
print(dataset)


dataset['train'] = dataset['train'].filter(lambda x: len(x['text'].split()) <= 2500)
dataset['train'] = dataset['train'].filter(lambda x: len(x['text'].split()) >= 200)
dataset['train'] = dataset['train'].filter(lambda x: len(x['summary'].split()) >= 40)

print(dataset)

split_train_temp = dataset["train"].train_test_split(test_size=0.4, seed=42)

split_valid_test = split_train_temp["test"].train_test_split(test_size=0.5, seed=42)

dataset_split = DatasetDict({
    "train": split_train_temp["train"],        
    "validation": split_valid_test["train"],      
    "test": split_valid_test["test"]              
})

print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 5000
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 4426
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 2655
    })
    validation: Dataset({
        features: ['text', 'summary'],
        num_rows: 885
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 886
    })
})


In [3]:
dataset_split.save_to_disk("dataset_split")

Saving the dataset (0/1 shards):   0%|          | 0/2655 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/885 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/886 [00:00<?, ? examples/s]